In [1]:
import io

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
data = pd.read_csv('Twitter_Data.csv')

In [2]:
data.drop(data[data['clean_text'].isna()].index, inplace=True)
data.drop(data[data['category'].isna()].index, inplace=True)

sentiment_map={-1:'negative',1:'positive',0:'neutral'}
data.insert(2,'sentiment',[sentiment_map[s] for s in data.category],True)

In [3]:
reviews = np.array(data['clean_text'])[:]
labels = np.array(data['sentiment'])[:]

In [4]:
# Data Cleaning
# lowercase, remove url, html, punctuations, retweet
import numpy.linalg._umath_linalg
import preprocess_kgptalkie as ps

data['clean_text'] = data['clean_text'].apply(lambda x: x.lower())
data['clean_text'] = data['clean_text'].apply(lambda x: ps.remove_urls(x))
data['clean_text'] = data['clean_text'].apply(lambda x: ps.remove_html_tags(x))
data['clean_text'] = data['clean_text'].apply(lambda x: ps.remove_special_chars(x))
data['clean_text'] = data['clean_text'].apply(lambda x: ps.remove_rt(x))

import re
from string import punctuation

def preProcessor(Tweet):
   
    text = re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w\-\.,@?^=%&:/\+#]*[\w\-\@?^=%&/\+#])?', ' ', Tweet)

    text = re.sub(r'<.*?>', ' ', text)

    text = re.sub(r'[^a-zA-Z\s]', '', text)
  
    text = re.sub(r'\bRT\b', '', text)
 
    text = text.lower()
    
    return text

data['clean_text'] = data['clean_text'].apply(preProcessor)


c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\preprocess_kgptalkie\utils.py:170: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(x, 'lxml').get_text().strip()


In [5]:
from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(text_counts,data['sentiment'],test_size=0.3)
x_train, x_test, y_train, y_test = train_test_split(data['clean_text'],data['sentiment'],test_size=0.3)

In [14]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from nltk.corpus import stopwords
from pyswarm import pso 

stop_words = stopwords.words('english')

def objective_function(params):
    max_depth, min_samples_split = params
    max_depth = max(1, int(round(max_depth)))
    min_samples_split = max(1, int(round( min_samples_split)))
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=5000, stop_words=stop_words)),
        ('clf', DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split))
    ])
    pipeline.fit(x_train, y_train)
    pred = pipeline.predict(x_test)
    accuracy = metrics.accuracy_score(y_test, pred)
    return -accuracy  


lower_bound = [1, 2] 
upper_bound = [10, 3]  

best_params, _ = pso(objective_function, lower_bound, upper_bound)

max_depth, min_samples_split = best_params
max_depth = max(1, int(round(max_depth)))
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, stop_words=stop_words)),
    ('clf', DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split))
])
pipeline.fit(x_train, y_train)
pred = pipeline.predict(x_test)
accuracy = metrics.accuracy_score(y_test, pred)
print("Optimized Decision Tree Accuracy:", accuracy)

   


Optimized Decision Tree Accuracy: 91.2
